In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np
import graphviz


Bad key "ytick.alignment" on line 268 in
/usr/share/matplotlib/mpl-data/stylelib/classic.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution

Bad key "xtick.alignment" on line 250 in
/usr/share/matplotlib/mpl-data/stylelib/classic.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
from collections import Counter
from imblearn.datasets import fetch_datasets
from sklearn import svm, tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report

In [3]:
df = pickle.load(open('dataframe.pickle', 'rb'))
print(df.dtypes)
df.head()

issuercountrycode                        int64
txvariantcode                           object
bin                                    float64
amount                                 float64
currencycode                            object
shoppercountrycode                      object
shopperinteraction                      object
cardverificationcodesupplied            object
cvcresponsecode                          int64
creationdate                    datetime64[ns]
accountcode                             object
mail_id                                  int64
ip_id                                    int64
card_id                                  int64
label                                    int64
creationdate_timestamp                 float64
dtype: object


,issuercountrycode,txvariantcode,bin,amount,currencycode,shoppercountrycode,shopperinteraction,cardverificationcodesupplied,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id,label,creationdate_timestamp
0,76,mccredit,530056.0,64800.0,MXN,MX,Ecommerce,True,0,2015-07-01,MexicoAccount,68370,111778,184798,1,1.435785e+09
1,76,mccredit,547046.0,44900.0,MXN,MX,Ecommerce,True,0,2015-07-02,MexicoAccount,101299,78749,151595,1,1.435805e+09
2,76,mccredit,528843.0,149900.0,MXN,MX,Ecommerce,True,0,2015-07-02,MexicoAccount,278604,70594,242142,1,1.435840e+09
3,76,mccredit,547146.0,109900.0,MXN,MX,Ecommerce,True,0,2015-07-03,MexicoAccount,47409,113648,181744,1,1.435903e+09
4,76,visaclassic,477291.0,89900.0,MXN,MX,Ecommerce,True,0,2015-07-08,MexicoAccount,205501,83553,97271,1,1.436373e+09


In [4]:
# TODO: to verify, does this dataframe takes into account Refused transactions?
# How many entries in total ?
df.shape[0]

236698

In [5]:
# How many of those are fraudolent and how many are safe ?
s = df.label.value_counts()
print s
print s[1]/float(s[0])

0    236353
1       345
Name: label, dtype: int64
0.00145968107026


In [6]:
# TODO clean and filter only the interesting features, this is just an example:
df_interesting = df[['label','ip_id','issuercountrycode']]
df_interesting = df_interesting.sample(frac=1).reset_index(drop=True)
df_interesting.head()

,label,ip_id,issuercountrycode
0,0,129025,40
1,0,135822,40
2,0,143351,97
3,0,335561,40
4,0,192368,40


In [7]:
# # Convert categorical variable into dummy/indicator variables for learning
# df_encoded = pd.get_dummies(df_interesting)
# df_encoded = df_encoded.sample(frac=1).reset_index(drop=True)

# df_encoded.head()

In [ ]:
# # Creating training and test sets
# training_features, test_features, \
# training_target, test_target, = train_test_split(df_encoded.drop(['label'], axis=1),
#                                                df_encoded['label'],
#                                                test_size = .1,
#                                                random_state=12)
X = np.array(df_interesting.drop(['label'], axis=1))
Y = np.array(df_interesting['label'])
print X.shape
print Y.shape

sm = SMOTE(random_state=12)
svc = svm.SVC()
dtc = tree.DecisionTreeClassifier(max_depth=3)
svc_set = []
dtc_set = []
dot_data_set = []

kf = KFold(n_splits=10) #Already shuffled
for train_index, test_index in kf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    X_train, Y_train = sm.fit_sample(X_train, Y_train)

    #print("X train SIZE: " + str(len(X_train)) + " Y train size: " + str(len(Y_train)))
    #print("X test SIZE: " + str(len(X_test)) + " Y test size: " + str(len(Y_test)))
    dtc = dtc.fit(X_train, Y_train)
    dtc_set.append(dtc)
    print("DTC SCORE: " + str(dtc.score(X_test, Y_test)))
    
    dot_data = tree.export_graphviz(dtc, out_file=None,
                               feature_names=list(df_interesting.columns.values[1:]),
                               class_names=['benign','fraud'],filled=True, rounded=True,  
                               special_characters=True
                              ) 
    dot_data_set.append(dot_data)
    
    svc.fit(X_train, Y_train)
    svc_set.append(svc)
    print("SVC SCORE: " + str(svc.score(X_test, Y_test)))
    

(236698, 2)
(236698,)
DTC SCORE: 0.849683143219


In [ ]:
graph = graphviz.Source(dot_data_set[0]) 
graph

In [ ]:
# print(training_features.shape)
# print(training_target.shape)

In [ ]:
# # SMOTE analysis (note, this is done after sampling)
# sm = SMOTE(random_state=12)
# x_res, y_res = sm.fit_sample(training_features, training_target)
# print training_target.value_counts()
# print np.bincount(y_res)
